In [39]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sn
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import warnings

In [40]:
# Suppress UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df_trips = pd.read_csv('trips_1.5M.csv')
df_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,1,2015-06-03 10:41:50,2015-06-03 11:08:33,1,2.80,-73.952888,40.776814,1,N,-73.979034,40.756611,1,17.5,0.0,0.5,4.57,0.0,22.87
1,2,2015-02-14 19:38:55,2015-02-14 20:00:59,1,2.79,-73.995827,40.725353,1,N,-73.984314,40.757389,2,15.0,0.0,0.5,0.00,0.0,15.80
2,2,2015-04-21 15:21:04,2015-04-21 15:33:38,1,1.75,-73.951920,40.769421,1,N,-73.956421,40.787140,2,9.5,0.0,0.5,0.00,0.0,10.30
3,2,2015-01-29 18:58:54,2015-01-29 19:04:46,3,0.83,-74.009026,40.715710,1,N,-74.003418,40.723152,1,5.5,1.0,0.5,1.46,0.0,8.76
4,1,2015-06-05 10:47:29,2015-06-05 10:58:00,3,0.80,-73.985909,40.756176,1,N,-73.988358,40.747398,2,7.0,0.0,0.5,0.00,0.0,7.80


# Memory usage optimization

In [3]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int64  
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int64  
 4   trip_distance          1500000 non-null  float64
 5   pickup_longitude       1500000 non-null  float64
 6   pickup_latitude        1500000 non-null  float64
 7   RateCodeID             1500000 non-null  int64  
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float64
 10  dropoff_latitude       1500000 non-null  float64
 11  payment_type           1500000 non-null  int64  
 12  fare_amount            1500000 non-null  float64
 13  extra                  1500000 non-null  float64
 14  mta_tax           

In [4]:
# To reduce memory usage, we will limit the datatypes of the columns
df_trips[['VendorID', 'passenger_count', 'RateCodeID', 'payment_type']] = df_trips[['VendorID', 'passenger_count', 'RateCodeID', 'payment_type']].astype('int8')
df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount' ]] = df_trips[['fare_amount','dropoff_longitude', 'dropoff_latitude', 'pickup_longitude', 'pickup_latitude', 'extra', 'mta_tax', 'tip_amount','tolls_amount']].astype('float16')
df_trips['total_amount'] = df_trips['total_amount'].astype('float32')
df_trips['trip_distance'] = df_trips['trip_distance'].astype('float16')
df_trips.info()

c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 18 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   VendorID               1500000 non-null  int8   
 1   tpep_pickup_datetime   1500000 non-null  object 
 2   tpep_dropoff_datetime  1500000 non-null  object 
 3   passenger_count        1500000 non-null  int8   
 4   trip_distance          1500000 non-null  float16
 5   pickup_longitude       1500000 non-null  float16
 6   pickup_latitude        1500000 non-null  float16
 7   RateCodeID             1500000 non-null  int8   
 8   store_and_fwd_flag     1500000 non-null  object 
 9   dropoff_longitude      1500000 non-null  float16
 10  dropoff_latitude       1500000 non-null  float16
 11  payment_type           1500000 non-null  int8   
 12  fare_amount            1500000 non-null  float16
 13  extra                  1500000 non-null  float16
 14  mta_tax           

# Data cleaning

## Adding the trip distance length in time units

In [5]:
# To predict the values, we will need to use the trip distances in time units, which we currently do not have
# We can do this based on the two datetime columns

# First, turning the two columns into datetime objects
df_trips["tpep_pickup_datetime"] = pd.to_datetime(df_trips["tpep_pickup_datetime"])
df_trips["tpep_dropoff_datetime"] = pd.to_datetime(df_trips["tpep_dropoff_datetime"])

# Then, we can calculate the time length of the trip
df_trips['time_length'] = (df_trips["tpep_dropoff_datetime"] - df_trips["tpep_pickup_datetime"]).dt.total_seconds() / 3600
df_trips = df_trips.drop(columns=['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'store_and_fwd_flag'])

df_trips

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,time_length
0,1,1,2.800781,-73.9375,40.78125,1,-74.0000,40.75000,1,17.5,0.0,0.5,4.570312,0.0,22.870001,0.445278
1,2,1,2.789062,-74.0000,40.71875,1,-74.0000,40.75000,2,15.0,0.0,0.5,0.000000,0.0,15.800000,0.367778
2,2,1,1.750000,-73.9375,40.78125,1,-73.9375,40.78125,2,9.5,0.0,0.5,0.000000,0.0,10.300000,0.209444
3,2,3,0.830078,-74.0000,40.71875,1,-74.0000,40.71875,1,5.5,1.0,0.5,1.459961,0.0,8.760000,0.097778
4,1,3,0.799805,-74.0000,40.75000,1,-74.0000,40.75000,2,7.0,0.0,0.5,0.000000,0.0,7.800000,0.175278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499995,1,1,0.700195,-74.0000,40.75000,1,-74.0000,40.75000,1,4.5,0.0,0.5,1.549805,0.0,6.850000,0.050278
1499996,1,1,0.799805,-74.0000,40.75000,1,-74.0000,40.75000,1,4.5,0.5,0.5,1.000000,0.0,6.800000,0.036944
1499997,1,1,2.000000,-73.9375,40.78125,1,-74.0000,40.75000,1,9.5,0.0,0.5,2.050781,0.0,12.350000,0.164722
1499998,2,1,1.959961,-74.0000,40.71875,1,-74.0000,40.75000,1,12.0,1.0,0.5,2.759766,0.0,16.559999,0.271111


## Limiting the variables' values

In [6]:
# To clean the data, we will remove rows with missing values and outliers by filtering the data on specific conditions (explained in the report)
print(f"Dataframe shape before cleaning: {df_trips.shape}")

df_trips_filtered = df_trips.loc[
    (df_trips['trip_distance'] < 21.0) & 
    (df_trips['trip_distance'] > 0.2) & 
    (df_trips['dropoff_longitude'] <= -73) & 
    (df_trips['dropoff_longitude'] > -75) & 
    (df_trips['dropoff_latitude'] >= 40) & 
    (df_trips['dropoff_latitude'] < 42) &
    (df_trips['pickup_longitude'] <= -73) & 
    (df_trips['pickup_longitude'] > -75) & 
    (df_trips['pickup_latitude'] >= 40) & 
    (df_trips['pickup_latitude'] < 42) &
    (df_trips['time_length'] > 0.02)
]

print(f"Dataframe shape after cleaning: {df_trips_filtered.shape}")

Dataframe shape before cleaning: (1500000, 16)
Dataframe shape after cleaning: (1450228, 16)


# Predictive model - distance cost matrix

## Data splitting

In [7]:
inputs = df_trips_filtered[['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude']]
labelDistance = df_trips_filtered['trip_distance']

x_trainDistance, x_testDistance, y_trainDistance, y_testDistance = train_test_split(inputs, labelDistance, random_state=69)

## Choice of regression model

### Comparison of possible regression models

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [11]:
# List of models for comparison
models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest Regression', RandomForestRegressor()),
    ('Decision Tree Regression', DecisionTreeRegressor()),
    ('K-Nearest Neighbors Regression', KNeighborsRegressor()),
    ('Gradient Boosting Regression', GradientBoostingRegressor()),
    ('AdaBoost Regression', AdaBoostRegressor()),
    ('Ridge Regression', Ridge()),
    ('Lasso Regression', Lasso()),
    ('Elastic Net Regression', ElasticNet()),
    ('Bayesian Regression', BayesianRidge()),
    ('Polynomial Regression', make_pipeline(PolynomialFeatures(degree=2), LinearRegression()))
]

# Loop through models and calculate MAE
for name, model in models:
    model.fit(x_trainDistance, y_trainDistance)
    y_pred = model.predict(x_testDistance)
    mae = mean_absolute_error(y_testDistance, y_pred)
    print(f"{name} MAE: {mae}")

c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)


Linear Regression MAE: 2.580078125
Random Forest Regression MAE: 0.781791186856506
Decision Tree Regression MAE: 0.7819644427577508


c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)


K-Nearest Neighbors Regression MAE: 0.9677734375
Gradient Boosting Regression MAE: 0.8587367929428915
AdaBoost Regression MAE: 1.4857064043582806
Ridge Regression MAE: 1.7682638782570488
Lasso Regression MAE: 2.1294448519861464
Elastic Net Regression MAE: 2.1294448519861464
Bayesian Regression MAE: 1.7683034922031813


c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)


Polynomial Regression MAE: 1.55078125


c:\Users\Damo\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)


### Comparison Result
As can be seen from above, the RF regression model is the most accurate. This will therefore be the model that will be kept for the rest of this code.

## Final model training

In [ ]:
# Train chosen model
finModelDistance = RandomForestRegressor()
finModelDistance.fit(x_trainDistance, y_trainDistance)

RandomForestRegressor()

## Predictions

### Importing depot and shop coordinates

In [15]:
# Depot coordinates and DF
depot_long = -73.941868
depot_lat = 40.725516
df_depot = pd.DataFrame({'id':'Depot', 'lat': [depot_lat], 'long': [depot_long]})

# Shops' coordinates and DF
df_shops = pd.read_csv('2015_shop_locations.csv')
df_shops = df_shops.drop(columns=['demand(kg)', 'stage'])

# Create new df that concatenates depot coordinates with df_shops
df_points = pd.concat([df_depot, df_shops], ignore_index=True)
df_points

,id,lat,long
0,Depot,40.725516,-73.941868
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,40.730837,-73.983194
2,ChIJU1XImaNZwokRutunetC8XeE,40.740454,-73.991268
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,40.694957,-73.982865
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,40.757522,-73.834352
...,...,...,...
216,ChIJ3wsoxKFZwokRRg8OvEjE0bc,40.736716,-73.986817
217,ChIJQRpR6Jj0wokRl9myJ19eEvU,40.850804,-73.867450
218,ChIJQYmJtx5awokR45_sopkXVNE,40.715137,-74.011088
219,ChIJh4MG8h9awokR1kDLslu2C3E,40.718249,-74.007346


### Distance cost matrix

In [41]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixDistance = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    lat1 = df_points.loc[df_points['id'] == id1, 'lat'].values[0]
    long1 = df_points.loc[df_points['id'] == id1, 'long'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        lat2 = df_points.loc[df_points['id'] == id2, 'lat'].values[0]
        long2 = df_points.loc[df_points['id'] == id2, 'long'].values[0]

        costMatrixDistance.loc[id1, id2] = finModelDistance.predict([[long1, lat1, long2, lat2]])[0]

costMatrixDistance

id,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,1.264695,3.289752,5.079517,4.125355,9.565742,6.672981,3.289752,10.293884,15.466988,13.913255,...,13.804221,10.293884,4.125355,15.535282,10.225981,5.079517,14.802674,3.289752,3.289752,3.899394
ChIJwTC7A55ZwokRPNX9g7ngbaI,3.652491,1.293245,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.55492,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,1.293245,1.293245,9.058113
ChIJU1XImaNZwokRutunetC8XeE,5.344794,2.462545,1.223754,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,1.223754,11.57671,2.462545,2.462545,8.268289
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.159567,2.779143,5.555309,1.35245,14.815639,8.422352,2.779143,10.851788,17.045178,10.390874,...,9.036164,10.851788,1.35245,14.318078,13.400577,5.555309,16.376429,2.779143,2.779143,9.556437
ChIJ5cPkuBtgwokRn55JgpGqjFA,8.44885,13.636841,10.780676,12.250608,2.086402,4.636728,13.636841,11.720111,10.178064,13.210052,...,17.468174,11.720111,12.266233,13.210052,6.411415,10.780676,15.669211,13.636841,13.636841,3.369102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,5.344794,2.462545,1.223754,5.815233,12.162233,2.746551,2.462545,5.210559,16.860678,15.701421,...,14.875472,5.210559,5.815233,17.368442,13.057781,1.223754,11.57671,2.462545,2.462545,8.268289
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.048655,12.831094,7.774905,8.075834,6.662563,5.516532,12.831094,3.592429,15.030873,19.568436,...,19.109784,3.592429,8.075834,19.568436,8.429137,7.774905,1.876177,12.831094,12.831094,15.773412
ChIJQYmJtx5awokR45_sopkXVNE,3.652491,1.293245,2.402347,3.398711,13.306269,5.361937,1.293245,7.911147,17.828954,13.407891,...,12.55492,7.911147,3.398711,17.193343,14.015943,2.402347,13.395202,1.293245,1.293245,9.058113


# Predictive model - duration cost matrix

## Data splitting

In [42]:
labelTime = df_trips_filtered['time_length']

x_trainTime, x_testTime, y_trainTime, y_testTime = train_test_split(inputs, labelTime, random_state=420)

## Final model training

In [43]:
# Train chosen model
finModelDuration = RandomForestRegressor()
finModelDuration.fit(x_trainTime, y_trainTime)

RandomForestRegressor()

## Predictions

### Importing depot and shop coordinates

In [44]:
# Depot coordinates and DF
depot_long = -73.941868
depot_lat = 40.725516
df_depot = pd.DataFrame({'id':'Depot', 'lat': [depot_lat], 'long': [depot_long]})

# Shops' coordinates and DF
df_shops = pd.read_csv('2015_shop_locations.csv')
df_shops = df_shops.drop(columns=['demand(kg)', 'stage'])

# Create new df that concatenates depot coordinates with df_shops
df_points = pd.concat([df_depot, df_shops], ignore_index=True)
df_points

,id,lat,long
0,Depot,40.725516,-73.941868
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,40.730837,-73.983194
2,ChIJU1XImaNZwokRutunetC8XeE,40.740454,-73.991268
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,40.694957,-73.982865
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,40.757522,-73.834352
...,...,...,...
216,ChIJ3wsoxKFZwokRRg8OvEjE0bc,40.736716,-73.986817
217,ChIJQRpR6Jj0wokRl9myJ19eEvU,40.850804,-73.867450
218,ChIJQYmJtx5awokR45_sopkXVNE,40.715137,-74.011088
219,ChIJh4MG8h9awokR1kDLslu2C3E,40.718249,-74.007346


### Duration cost matrix

In [ ]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixDuration = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    lat1 = df_points.loc[df_points['id'] == id1, 'lat'].values[0]
    long1 = df_points.loc[df_points['id'] == id1, 'long'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        lat2 = df_points.loc[df_points['id'] == id2, 'lat'].values[0]
        long2 = df_points.loc[df_points['id'] == id2, 'long'].values[0]

        costMatrixDuration.loc[id1, id2] = finModelDuration.predict([[long1, lat1, long2, lat2]])[0]

costMatrixDuration

# Predictive model - financial cost matrix

## Data splitting

In [ ]:
labelFinancial = df_trips_filtered['time_length']

x_trainFinancial, x_testFinancial, y_trainFinancial, y_testFinancial = train_test_split(inputs, labelFinancial, random_state=420)

## Final model training

In [ ]:
# Train chosen model
finModelFinancial = RandomForestRegressor()
finModelFinancial.fit(x_trainFinancial, y_trainFinancial)

RandomForestRegressor()

## Predictions

### Importing depot and shop coordinates

In [ ]:
# Depot coordinates and DF
depot_long = -73.941868
depot_lat = 40.725516
df_depot = pd.DataFrame({'id':'Depot', 'lat': [depot_lat], 'long': [depot_long]})

# Shops' coordinates and DF
df_shops = pd.read_csv('2015_shop_locations.csv')
df_shops = df_shops.drop(columns=['demand(kg)', 'stage'])

# Create new df that concatenates depot coordinates with df_shops
df_points = pd.concat([df_depot, df_shops], ignore_index=True)
df_points

,id,lat,long
0,Depot,40.725516,-73.941868
1,ChIJwTC7A55ZwokRPNX9g7ngbaI,40.730837,-73.983194
2,ChIJU1XImaNZwokRutunetC8XeE,40.740454,-73.991268
3,ChIJG7L-TLVbwokRT36uIrwz2Mo,40.694957,-73.982865
4,ChIJ5cPkuBtgwokRn55JgpGqjFA,40.757522,-73.834352
...,...,...,...
216,ChIJ3wsoxKFZwokRRg8OvEjE0bc,40.736716,-73.986817
217,ChIJQRpR6Jj0wokRl9myJ19eEvU,40.850804,-73.867450
218,ChIJQYmJtx5awokR45_sopkXVNE,40.715137,-74.011088
219,ChIJh4MG8h9awokR1kDLslu2C3E,40.718249,-74.007346


### Financial cost matrix

In [ ]:
# Predict the cost matrix using every couple of ids in df_points
## First create a new df for the cost matrix
costMatrixFinancial = pd.DataFrame(columns=df_points['id'], index=df_points['id'])

## Second fill the cost matrix with the predicted values
for id1 in df_points['id']:
    # Get the latitude and longitude for id1
    lat1 = df_points.loc[df_points['id'] == id1, 'lat'].values[0]
    long1 = df_points.loc[df_points['id'] == id1, 'long'].values[0]
    
    for id2 in df_points['id']:
        # Get the latitude and longitude for id2
        lat2 = df_points.loc[df_points['id'] == id2, 'lat'].values[0]
        long2 = df_points.loc[df_points['id'] == id2, 'long'].values[0]

        costMatrixFinancial.loc[id1, id2] = finModelFinancial.predict([[long1, lat1, long2, lat2]])[0]

costMatrixFinancial